In [1]:
%run ../../Utils/yp_utils.py

# Initial setup

In [2]:
paper_pmid = 23071506
paper_name = 'lockshon_kennedy_2012' 

In [3]:
datasets = pd.read_csv('extras/YeastPhenome_' + str(paper_pmid) + '_datasets_list.txt', sep='\t', header=None, names=['pmid', 'name'])

In [4]:
datasets.set_index('pmid', inplace=True)

# Load & process the data

In [5]:
original_data = pd.read_excel('raw_data/journal.pone.0045049.s009.XLS', sheet_name='PO inhib order', skiprows=1)

In [6]:
print('Original data dimensions: %d x %d' % (original_data.shape))

Original data dimensions: 212 x 8


In [7]:
original_data['orf'] = original_data['orf'].astype(str)

In [8]:
# Eliminate all white spaces & capitalize
original_data['orf'] = clean_orf(original_data['orf'])

In [9]:
# Translate to ORFs 
original_data['orf'] = translate_sc(original_data['orf'], to='orf')

In [10]:
# Make sure everything translated ok
t = looks_like_orf(original_data['orf'])
print(original_data.loc[~t,])

Empty DataFrame
Columns: [PO inh, OA inh, orf, position, gene, function, OA stim, POstim]
Index: []


In [11]:
original_data['PO inh'] = -original_data['PO inh']

In [12]:
((original_data['PO inh'] < 0) & (original_data['POstim'] > 0)).sum()

0

In [13]:
original_data['data'] = original_data[['PO inh','POstim']].sum(axis=1)

In [14]:
original_data.head()

,PO inh,OA inh,orf,position,gene,function,OA stim,POstim,data
index_input,,,,,,,,,
0,-3,0,YER093C-A,131f2,AIM11,function unknown,3,0,-3
1,-3,0,YLR425W,135b9,TUS1,Rho guanyl-nucleotide exchange factor activity,3,0,-3
2,-3,0,YJL095W,133h6,BCK1,MAP kinase kinase kinase activity,2,0,-3
3,-3,0,YLR337C,171e8,VRP1,actin binding,2,0,-3
4,-3,0,YMR021C,170a7,MAC1,specific RNA polymerase II transcription facto...,1,0,-3


In [15]:
original_data.set_index('orf', inplace=True)

In [16]:
original_data = original_data[['data']].copy()

In [17]:
original_data = original_data.groupby(original_data.index).mean()

# Prepare the final dataset

In [18]:
data = original_data.copy()

In [19]:
dataset_ids = [16611]

In [20]:
datasets = datasets.reindex(index=dataset_ids)

In [21]:
lst = [datasets.index.values, ['value']*datasets.shape[0]]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','data_type'])
data.columns = idx

In [22]:
data.head()

dataset_id,16611
data_type,value
orf,
YAL009W,0
YAL051W,-3
YAL055W,-3
YAR014C,0
YBL006C,-2


## Subset to the genes currently in SGD

In [23]:
genes = pd.read_csv(path_to_genes, sep='\t', index_col='id')
genes = genes.reset_index().set_index('systematic_name')
gene_ids = genes.reindex(index=data.index.values)['id'].values
num_missing = np.sum(np.isnan(gene_ids))
print('ORFs missing from SGD: %d' % num_missing)

ORFs missing from SGD: 0


In [24]:
data['gene_id'] = gene_ids
data = data.loc[data['gene_id'].notnull()]
data['gene_id'] = data['gene_id'].astype(int)
data = data.reset_index().set_index(['gene_id','orf'])

data.head()

,dataset_id,16611
,data_type,value
gene_id,orf,
7,YAL009W,0
48,YAL051W,-3
51,YAL055W,-3
66,YAR014C,0
94,YBL006C,-2


# Normalize

In [25]:
data.shape

(212, 1)

In [26]:
data_norm = normalize_phenotypic_scores(data, has_tested=False)

In [27]:
# Assign proper column names
lst = [datasets.index.values, ['valuez']*datasets.shape[0]]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','data_type'])
data_norm.columns = idx

In [28]:
data_norm[data.isnull()] = np.nan
data_all = data.join(data_norm)

data_all.head()

dataset_id      16611          
data_type       value    valuez
gene_id orf                    
7       YAL009W     0  0.066448
48      YAL051W    -3 -6.511906
51      YAL055W    -3 -6.511906
66      YAR014C     0  0.066448
94      YBL006C    -2 -4.319122

# Print out

In [29]:
for f in ['value','valuez']:
    df = data_all.xs(f, level='data_type', axis=1).copy()
    df.columns = datasets['name'].values
    df = df.droplevel('gene_id', axis=0)
    df.to_csv(paper_name + '_' + f + '.txt', sep='\t')

# Save to DB

In [30]:
from IO.save_data_to_db3 import *

In [31]:
save_data_to_db(data_all, paper_pmid)

  0%|          | 0/1 [00:00<?, ?it/s]

Deleting all datasets for PMID 23071506...
Inserting the new data...


100%|██████████| 1/1 [00:00<00:00,  1.83it/s]

Updating the data_modified_on field...
